# Consdb for LSSTCam, LSSTComCam, Latiss and LSSTComCamSim

- Creation date : 2025-04-28
- update : 2025-05-05 : Add Latiss ==> find data in 2022 - 2025
- update : 2025-05-07 : Search for LSSTComCamSim : https://usdf-rsp-dev.slac.stanford.edu/consdb/ and review all plots
- update : 2025-05-13 : Add Corentin filters on visits
- https://usdf-rsp-dev.slac.stanford.edu/consdb/
- Schemes for constdb : https://sdm-schemas.lsst.io/
- Documentation : https://consdb.lsst.io/index.html
- w_2025_17


In [ ]:
from lsst.summit.utils import ConsDbClient

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.dates as mdates

import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.cm as cmx
from matplotlib.colors import ListedColormap

import seaborn as sns

%matplotlib widget

from lsst.meas.algorithms.installGaussianPsf import FwhmPerSigma

from tqdm.notebook import tqdm

In [ ]:
# xx-small
# x-small
# small
# medium
# large
# x-large
# xx-large

plt.rcParams["figure.figsize"] = (10, 6)
plt.rcParams["axes.labelsize"] = "x-large"
plt.rcParams["axes.titlesize"] = "x-large"
plt.rcParams["xtick.labelsize"] = "x-large"
plt.rcParams["ytick.labelsize"] = "x-large"

In [ ]:
from astropy.table import Table, join
from astropy.time import Time
from astropy.coordinates import SkyCoord
from astropy import coordinates
import astropy.coordinates as coord
import astropy.units as u

In [ ]:
import lsst.geom as geom
import lsst
from lsst.geom import Angle

In [ ]:
# https://pipelines.lsst.io/modules/lsst.geom/getting-started.html
func_degToRad = lambda x: Angle(x, lsst.geom.degrees).asRadians()
func_zendtoAirmass = lambda x: 1.0 / np.cos(func_degToRad(x))
# func_wrap = lambda x : Angle(x,lsst.geom.radians).wrap(180.*lsst.geom.degrees)

In [ ]:
def ra_to_mollweide(ra_rad):
    """
    Transforme une ascension droite (RA en radians) pour une projection Mollweide :
    - RA doit être d'abord dans [0, 2π]
    - Puis ramené dans [-π, +π] pour correspondre au domaine Mollweide
    - Puis inversé pour que RA croisse vers la gauche sur la carte

    Paramètre:
    - ra_rad : array ou scalaire de RA en radians

    Retourne:
    - RA transformé pour projection Mollweide

    Discussion avec ChatGPT verifiée le 28/04/2025
    """

    # 1) Assurer que RA est entre 0 et 2π
    ra_rad = np.remainder(ra_rad + 2 * np.pi, 2 * np.pi)

    # 2) Remettre RA entre -π et +π
    ra_rad[ra_rad > np.pi] -= 2 * np.pi

    # 3) Inverser RA pour que RA croisse vers la gauche
    return -ra_rad

## Configuration

### Choose instrument

In [ ]:
# instrument = "LSSTCam"
#instrument = "LSSTComCam"
instrument = "LATISS" # data since 20221001
# instrument = "LSSTComCamSim"

In [ ]:
os.environ["no_proxy"] += ",.consdb"

In [ ]:
url = "http://consdb-pq.consdb:8080/consdb"

In [ ]:
consdb = ConsDbClient(url)

https://sdm-schemas.lsst.io/cdb_lsstcomcam.html#exposure

In [ ]:
# Query both consDB tables
# exposure = consdb.query("SELECT * FROM cdb_lsstcam.exposure WHERE science_program = 'BLOCK-351'")
# visits = consdb.query("SELECT * FROM cdb_lsstcam.visit1 WHERE science_program = 'BLOCK-351'")
# visits_ql = consdb.query("SELECT * FROM cdb_lsstcam.visit1_quicklook")
# visits_ql = consdb.query("SELECT * FROM cdb_lsstcam.visit1")

if instrument == "LSSTCam":
    exposures = consdb.query("SELECT * FROM cdb_lsstcam.exposure WHERE day_obs >= 20250415")
    visits = consdb.query("SELECT * FROM cdb_lsstcam.visit1 WHERE day_obs >= 20250415")

elif instrument == "LSSTComCam":
    exposures = consdb.query("SELECT * FROM cdb_lsstcomcam.exposure WHERE day_obs >= 20241021")
    visits = consdb.query("SELECT * FROM cdb_lsstcomcam.visit1 WHERE day_obs >= 20241021")

elif instrument == "LATISS":
    exposures = consdb.query("SELECT * FROM cdb_latiss.exposure WHERE day_obs >= 20220101")
    visits = consdb.query("SELECT * FROM cdb_latiss.visit1 WHERE day_obs >= 20220101")

elif instrument == "LSSTComCamSim":
    # 3 nights of simulation
    # collection1 = 'LSSTComCamSim/runs/nightlyvalidation/20240402/d_2024_03_29/DM-43612'
    # collection2 = 'LSSTComCamSim/runs/nightlyvalidation/20240403/d_2024_03_29/DM-43612'
    # collection3 = 'LSSTComCamSim/runs/nightlyvalidation/20240404/d_2024_03_29/DM-43612'
    # exposure = consdb.query("SELECT * FROM cdb_lsstcomcamsim.exposure WHERE day_obs >= 202404002")
    # visits = consdb.query("SELECT * FROM cdb_lsstcomcamsim.visit1 WHERE day_obs >= 202404002")
    exposures = consdb.query("SELECT * FROM cdb_lsstcomcamsim.exposure WHERE day_obs >= 20240402")
    visits = consdb.query("SELECT * FROM cdb_lsstcomcamsim.visit1 WHERE day_obs >= 20240402")


else:
    raise (f"This notebook has not implemented a SQL query to ConsDB for the {instrument}")


# Join using astropy's join function on 'visit_id'
# exposure_join = exposure.rename_column("exposure_id", "visit_id")
# merged_exposure = join(exposure, visits, keys="visit_id", join_type="inner")
# merged_visits = join(visits, visits_ql, keys="visit_id", join_type="inner")

# Display or use the merged table
# print(merged_visits)

In [ ]:
print(visits.columns)

In [ ]:
print(exposures.columns)

In [ ]:
df_exposures = exposures.to_pandas()
df_visits = visits.to_pandas()

In [ ]:
df_exposures.head()

In [ ]:
df_visits.head()

In [ ]:
df_visits.dropna(subset=["zenith_distance_start", "exp_midpt_mjd"])[
    ["zenith_distance_start", "exp_midpt_mjd"]
]

In [ ]:
df_visits = df_visits.dropna(subset=["zenith_distance_start", "exp_midpt_mjd"])

In [ ]:
df_visits[["day_obs", "zenith_distance_start", "exp_midpt_mjd"]]

### Don't see the difference between the visits and the exposures !

## Explore the visit info in consdb

### Observation dates

In [ ]:
df_visits["day_obs"].unique()

In [ ]:
if instrument == "LSSTComCamSim":
    # selection_cut = (df_visits["day_obs"] <= 20240404)
    # df_visits = df_visits[selection_cut]
    # exclude nodate no zenith value
    # df_visits = df_visits.dropna(subset=['exp_midpt_mjd'])
    df_visits = df_visits.dropna(subset=["zenith_distance_start", "exp_midpt_mjd"])

    # df_visits.dropna(subset=['zenith_distance_start','exp_midpt_mjd'],inplace=True)
    dt = [Time(mjd, format="mjd", scale="utc").isot for mjd in df_visits["exp_midpt_mjd"].values]
    # convert the MJD time in ISO 8601
    df_visits["exp_midpt"] = dt

    df_visits["airmass"] = df_visits["zenith_distance_start"].apply(lambda x: 1 / np.cos(x / 180.0 * np.pi))

In [ ]:
df_visits

### Filters

In [ ]:
print(df_visits["physical_filter"].unique(), df_visits["band"].unique())

### Remove bad filters

In [ ]:
df_visits = df_visits[
    ~(
        (df_visits["physical_filter"] == "other")
        | (df_visits["physical_filter"] == "none")
        | (df_visits["physical_filter"] == "other:pinhole")
        | ("pinhole" in df_visits["physical_filter"])
        | ("cyl_lens" in df_visits["physical_filter"])
        | (df_visits["band"] == "none")
        | (df_visits["band"] == "other:pinhole")
    )
]

### Remove rows  with bad ra and dec

In [ ]:
df_visits = df_visits.dropna(subset=["s_ra", "s_dec"])

### Science program and observation reason

In [ ]:
df_visits["science_program"].unique()

In [ ]:
df_visits["observation_reason"].unique()

### Check bands

In [ ]:
df_visits.band.unique()

### Time exposure

In [ ]:
# add missing column in table
if instrument == "LSSTComCamSim":
    df_visits["exp_time"] = 30.0

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 4))
ax.hist(df_visits["exp_time"].values, bins=50, facecolor="b")
plt.show()

## Region in sky

In [ ]:
dict_b_to_col = {
    "u": "b",
    "g": "g",
    "r": "r",
    "i": "orange",
    "z": "magenta",
    "y": "purple",
    "other": "grey",
    "none": "k",
    None: "k",
    "other:pinhole": "grey",
    "EMPTY": "grey",
    "white": "grey",
}

In [ ]:
col = df_visits["band"].map(lambda b: dict_b_to_col[b]).values

In [ ]:
col

In [ ]:
palette_spectral = sns.color_palette("Spectral_r", as_cmap=True)
cmap_time = ListedColormap(sns.color_palette("Spectral_r", df_visits.size))
dt = df_visits["obs_start_mjd"].values - df_visits["obs_start_mjd"].min()
dtmin = dt.min()
dtmax = dt.max()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12, 6))
im = df_visits.plot.scatter(x="s_ra", y="s_dec", ax=ax, c=dt, cmap=cmap_time)
ax.grid()
ax.set_aspect("equal")
# cbar = plt.colorbar(im, orientation='horizontal', label='time(days)')
plt.show()

In [ ]:
palette_spectral = sns.color_palette("Spectral_r", as_cmap=True)

In [ ]:
cmap_time = ListedColormap(sns.color_palette("Spectral", df_visits.size))

In [ ]:
dt = df_visits["obs_start_mjd"].values - df_visits["obs_start_mjd"].min()

In [ ]:
dtmin = dt.min()
dtmax = dt.max()

In [ ]:
ra_g = df_visits["s_ra"].apply(func_degToRad)
dec_g = df_visits["s_dec"].apply(func_degToRad)

# ra_g = coordinates.Angle(df_visits['s_ra'].values,unit="deg").radian
# dec_g = coordinates.Angle(df_visits['s_dec'].values,unit="deg").radian

In [ ]:
# Galactic plane
gal_long = np.linspace(-180.0, 180, 360)
gal_lat = np.zeros((360))
coordinates_galactic_planes = SkyCoord(l=gal_long * u.degree, b=gal_lat * u.degree, frame="galactic")
gp_radec = coordinates_galactic_planes.transform_to("icrs")

# for galactic plane
# gp_radec.ra : 0., 360.
# gp_radec.dec : -90, 90
# Old method
# gp_ra_toplot = coordinates.Angle(gp_radec.ra.degree*u.degree)
# gp_ra_toplot = gp_ra_toplot.wrap_at(180*u.degree)
# gp_ra_toplot -180, 180

# new method in radian with ra_to_mollweide
gp_ra_toplot = ra_to_mollweide(gp_radec.ra.radian)
gp_dec_toplot = gp_radec.dec.radian

In [ ]:
ra = coordinates.Angle(df_visits["s_ra"].values, unit="deg").radian
dec = coordinates.Angle(df_visits["s_dec"].values, unit="deg").radian

In [ ]:
# plot
# x arg in rad must be (-2pi,2pi), y arg in rad  must be in ( -pi,pi )
fig = plt.figure(figsize=(14, 8))
# ax = fig.add_subplot(111, projection="aitoff")
ax = fig.add_subplot(111, projection="mollweide")

# galactic plane
# ax.scatter(ra_to_mollweide(gp_ra_toplot.radian), gp_radec.dec.radian, c="r", label="Galactic Plane", s=1)
# galactic plane
ax.scatter(gp_ra_toplot, gp_dec_toplot, c="r", label="Galactic Plane", s=1)

# ax.scatter(ra-np.pi,dec,marker='+',c=all_colors_g)
# im = ax.scatter(ra_g.values-np.pi,dec_g.values,marker='+',s=50,lw=3,c=dt,cmap=palette_spectral)
im = ax.scatter(
    ra_to_mollweide(ra_g.values), dec_g.values, marker="+", s=50, lw=2, c=dt, cmap=palette_spectral
)

ax.legend()
ax.grid()

cbar = plt.colorbar(im, orientation="horizontal", label="time(days)")
plt.suptitle(instrument, fontsize=30, fontweight="bold")

plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Coordonnées des Deep Drilling Fields (RA en degrés, Dec en degrés)
ddf_names = ["XMM-LSS", "COSMOS", "ECDFS", "ELAIS-S1", "EDFS_a", "EDFS_b"]
ddf_ra_deg = np.array([35.57, 150.11, 52.98, 9.45, 58.90, 63.60])  # approximatif
ddf_dec_deg = np.array([-4.82, 2.23, -28.12, -44.02, -49.32, -47.60])  # approximatif


# Conversion en radians et adaptation pour Mollweide projection
ddf_ra_rad = np.radians(ddf_ra_deg)
ddf_dec_rad = np.radians(ddf_dec_deg)
# Décalage pour Mollweide (RA=0h au centre), donc -π à π
# ddf_ra_rad = np.remainder(ddf_ra_rad + 2*np.pi, 2*np.pi)  # force RA entre 0 et 2pi
# ddf_ra_rad[ddf_ra_rad > np.pi] -= 2*np.pi  # force entre -pi et +pi
# ddf_ra_rad = -ddf_ra_rad  # reverse l'axe des RA
ddf_ra_rad = ra_to_mollweide(ddf_ra_rad)

# --- Ton plot existant ---
fig = plt.figure(figsize=(14, 8))
ax = fig.add_subplot(111, projection="mollweide")

# Exemple de fond existant
# ax.scatter(ra_to_mollweide(gp_ra_toplot.radian), gp_radec.dec.radian, c="r", label="Galactic Plane", s=1)
# galactic plane
ax.scatter(gp_ra_toplot, gp_dec_toplot, c="r", label="Galactic Plane", s=1)
# im = ax.scatter(ra_g.values - np.pi, dec_g.values, marker='+', s=50, lw=3, c=dt, cmap=palette_spectral)
im = ax.scatter(
    ra_to_mollweide(ra_g.values), dec_g.values, marker="+", s=50, lw=2, c=dt, cmap=palette_spectral
)

# Ajout des Deep Drilling Fields
for i in range(len(ddf_names)):
    ax.plot(ddf_ra_rad[i], ddf_dec_rad[i], "o", markersize=10, color="blue", markeredgecolor="black")
    ax.text(
        ddf_ra_rad[i] + 0.05,
        ddf_dec_rad[i] + 0.05,
        ddf_names[i],
        fontsize=12,
        color="blue",
        ha="left",
        va="bottom",
        weight="bold",
    )

ax.legend()
ax.grid()

cbar = plt.colorbar(im, orientation="horizontal", label="time (days)")
plt.suptitle(instrument, fontsize=30, fontweight="bold")

plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Coordonnées des Deep Drilling Fields (RA en degrés, Dec en degrés)
ddf_names = ["XMM-LSS", "COSMOS", "ECDFS", "ELAIS-S1", "EDFS_a", "EDFS_b"]
ddf_ra_deg = np.array([35.57, 150.11, 52.98, 9.45, 58.90, 63.60])  # approximatif
ddf_dec_deg = np.array([-4.82, 2.23, -28.12, -44.02, -49.32, -47.60])  # approximatif

# Conversion en radians et adaptation pour Mollweide projection
ddf_ra_rad = np.radians(ddf_ra_deg)
ddf_dec_rad = np.radians(ddf_dec_deg)
# ddf_ra_rad = np.remainder(ddf_ra_rad + 2*np.pi, 2*np.pi)
# ddf_ra_rad[ddf_ra_rad > np.pi] -= 2*np.pi
# ddf_ra_rad = -ddf_ra_rad
ddf_ra_rad = ra_to_mollweide(ddf_ra_rad)

# Rayon du cercle en radians (~3.5 degrés)
radius_deg = 3.5
radius_rad = np.radians(radius_deg)

# --- Ton plot existant ---
fig = plt.figure(figsize=(14, 8))
ax = fig.add_subplot(111, projection="mollweide")

# Galactic plane
# ax.scatter(ra_to_mollweide(gp_ra_toplot.radian), gp_radec.dec.radian, c="r", label="Galactic Plane", s=1)
# galactic plane
ax.scatter(gp_ra_toplot, gp_dec_toplot, c="r", label="Galactic Plane", s=1)
# im = ax.scatter(ra_g.values - np.pi, dec_g.values, marker='+', s=50, lw=3, c=dt, cmap=palette_spectral)
im = ax.scatter(
    ra_to_mollweide(ra_g.values), dec_g.values, marker="+", s=50, lw=2, c=dt, cmap=palette_spectral
)

# Couleur choisie
ddf_color = "navy"  # bleu foncé

# Ajout des Deep Drilling Fields
for i in range(len(ddf_names)):
    # Marqueur central
    ax.plot(ddf_ra_rad[i], ddf_dec_rad[i], "o", markersize=10, lw=2, color=ddf_color, markeredgecolor="black")
    ax.text(
        ddf_ra_rad[i] + 0.05,
        ddf_dec_rad[i] + 0.05,
        ddf_names[i],
        fontsize=12,
        color=ddf_color,
        ha="left",
        va="bottom",
        weight="bold",
    )

    # Cercle autour
    theta = np.linspace(0, 2 * np.pi, 100)
    circle_ra = ddf_ra_rad[i] + radius_rad * np.cos(theta)
    circle_dec = ddf_dec_rad[i] + radius_rad * np.sin(theta)
    ax.plot(circle_ra, circle_dec, color=ddf_color, linestyle="--", lw=1)

ax.legend()
ax.grid()

cbar = plt.colorbar(im, orientation="horizontal", label="time (days)")
plt.suptitle(instrument, fontsize=30, fontweight="bold")

# --- Ajouter aussi le LMC ---
# Coordonnées du Grand Nuage de Magellan (LMC)
lmc_ra_deg = 80.8939
lmc_dec_deg = -69.7561

# Conversion en radians
lmc_ra_rad = np.radians(lmc_ra_deg)
lmc_dec_rad = np.radians(lmc_dec_deg)

# Ajustement Mollweide
lmc_ra_rad = np.remainder(lmc_ra_rad + 2 * np.pi, 2 * np.pi)
if lmc_ra_rad > np.pi:
    lmc_ra_rad -= 2 * np.pi
lmc_ra_rad = -lmc_ra_rad
# lmc_ra_rad = ra_to_mollweide([lmc_ra_rad])[0]

# Ajouter LMC au plot
ax.plot(
    lmc_ra_rad,
    lmc_dec_rad,
    marker="*",
    color="darkorange",
    markersize=15,
    markeredgecolor="black",
    label="LMC",
)
ax.text(
    lmc_ra_rad + 0.05,
    lmc_dec_rad + 0.05,
    "LMC",
    fontsize=10,
    color="darkorange",
    ha="left",
    va="bottom",
    weight="bold",
)


plt.show()

## Next convert back df_visits into astropy table

In [ ]:
visits = Table.from_pandas(df_visits)

In [ ]:
# visits

## Time dependence

In [ ]:
time = Time(visits["exp_midpt"])

In [ ]:
print("tmin = ", time.min(), " | tmax = ", time.max())

In [ ]:
# dattim = [ Time(dt) for dt in df_visits["exp_midpt"].values]

In [ ]:
# time.value

In [ ]:
# df_visits[["exp_midpt","airmass"]]

In [ ]:
# df_visits["exp_midpt"]

In [ ]:
# Conversion MJD → datetime (UTC)
df_visits["exp_midpt"] = pd.to_datetime(df_visits["exp_midpt"], format="mixed")

### Airmass vs Time

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12, 4), layout="constrained")

df_visits.plot(x="exp_midpt", y="airmass", ax=ax, marker=".", lw=0, color="b")
# plt.plot(df_visits["exp_midpt"].values,df_visits["airmass"].values,marker="+",c="b",lw=0)
# plt.plot(df_visits["exp_midpt"].values,df_visits["airmass"].values,marker="+",c="b",lw=0)
# Set x-axis to show dates
# plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d %H:%M:%S'))
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter("%Y-%m-%dT%H"))
plt.gca().xaxis.set_major_locator(mdates.AutoDateLocator())

# Rotate and format x-axis labels for readability
# ax.set_xticks(rotation=45, ha="right")
ax.set_ylim(2.5, 0.9)
ax.set_xlabel("Time")
ax.set_ylabel("Airmass")
ax.set_title(f"Time vs Airmass")
ax.grid()

plt.show()

### Airmass vs Temperature

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12, 4), layout="constrained")

df_visits.plot(x="exp_midpt", y="air_temp", ax=ax, marker=".", lw=0, color="b")
# plt.plot(df_visits["exp_midpt"].values,df_visits["airmass"].values,marker="+",c="b",lw=0)
# plt.plot(df_visits["exp_midpt"].values,df_visits["airmass"].values,marker="+",c="b",lw=0)
# Set x-axis to show dates
# plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d %H:%M:%S'))
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter("%Y-%m-%dT%H"))
plt.gca().xaxis.set_major_locator(mdates.AutoDateLocator())

# Rotate and format x-axis labels for readability
# ax.set_xticks(rotation=45, ha="right")
# ax.set_ylim(2,0.9)
ax.set_xlabel("Time")
ax.set_ylabel("Temperature")
ax.set_title(f"Time vs Temperature")
ax.grid()

plt.show()

### Airmass vs Pressure

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12, 4), layout="constrained")

df_visits.plot(x="exp_midpt", y="pressure", ax=ax, marker=".", lw=0, color="b")
# plt.plot(df_visits["exp_midpt"].values,df_visits["airmass"].values,marker="+",c="b",lw=0)
# plt.plot(df_visits["exp_midpt"].values,df_visits["airmass"].values,marker="+",c="b",lw=0)
# Set x-axis to show dates
# plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d %H:%M:%S'))
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter("%Y-%m-%dT%H"))
plt.gca().xaxis.set_major_locator(mdates.AutoDateLocator())

# Rotate and format x-axis labels for readability
# ax.set_xticks(rotation=45, ha="right")
# ax.set_ylim(2,0.9)
ax.set_xlabel("Time")
ax.set_ylabel("Pressure")
ax.set_title(f"Time vs Pressure")
ax.grid()

plt.show()

###  Humidity vs time

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12, 4), layout="constrained")

df_visits.plot(x="exp_midpt", y="humidity", ax=ax, marker=".", lw=0, color="b")
# plt.plot(df_visits["exp_midpt"].values,df_visits["airmass"].values,marker="+",c="b",lw=0)
# plt.plot(df_visits["exp_midpt"].values,df_visits["airmass"].values,marker="+",c="b",lw=0)
# Set x-axis to show dates
# plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d %H:%M:%S'))
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter("%Y-%m-%dT%H"))
plt.gca().xaxis.set_major_locator(mdates.AutoDateLocator())

# Rotate and format x-axis labels for readability
# ax.set_xticks(rotation=45, ha="right")
# ax.set_ylim(2,0.9)
ax.set_xlabel("Time")
ax.set_ylabel("Humidity")
ax.set_title(f"Humidity vs Time")
ax.grid()

plt.show()

### Windspeed vs time

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12, 4), layout="constrained")

df_visits.plot(x="exp_midpt", y="wind_speed", ax=ax, marker=".", lw=0, color="b")
# plt.plot(df_visits["exp_midpt"].values,df_visits["airmass"].values,marker="+",c="b",lw=0)
# plt.plot(df_visits["exp_midpt"].values,df_visits["airmass"].values,marker="+",c="b",lw=0)
# Set x-axis to show dates
# plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d %H:%M:%S'))
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter("%Y-%m-%dT%H"))
plt.gca().xaxis.set_major_locator(mdates.AutoDateLocator())

# Rotate and format x-axis labels for readability
# ax.set_xticks(rotation=45, ha="right")
# ax.set_ylim(2,0.9)
ax.set_xlabel("Time")
ax.set_ylabel("Wind speed")
ax.set_title(f"Wind speed vs Time")
ax.grid()

plt.show()

### Winds direction vs time

In [ ]:
# df_visits["wind_dir_shifted"] = np.remainder(df_visits["wind_dir"]+180.,360)- 180
df_visits["wind_dir_shifted"] = ((df_visits["wind_dir"] + 180) % 360) - 180

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12, 4), layout="constrained")

# df_visits.plot(x="exp_midpt",y="wind_dir",ax=ax,marker=".",lw=0,color="b")
df_visits.plot(x="exp_midpt", y="wind_dir_shifted", ax=ax, marker=".", lw=0, color="r")
# plt.plot(df_visits["exp_midpt"].values,df_visits["airmass"].values,marker="+",c="b",lw=0)
# plt.plot(df_visits["exp_midpt"].values,df_visits["airmass"].values,marker="+",c="b",lw=0)
# Set x-axis to show dates
# plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d %H:%M:%S'))
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter("%Y-%m-%dT%H"))
plt.gca().xaxis.set_major_locator(mdates.AutoDateLocator())

# Rotate and format x-axis labels for readability
# ax.set_xticks(rotation=45, ha="right")
# ax.set_ylim(2,0.9)
ax.set_xlabel("Time")
ax.set_ylabel("Wind dir")
ax.set_title(f"Wind direction vs Time")
ax.grid()

plt.show()

### Dimm_seeing vs time

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12, 4), layout="constrained")

df_visits.plot(x="exp_midpt", y="dimm_seeing", ax=ax, marker=".", lw=0, color="b")
# plt.plot(df_visits["exp_midpt"].values,df_visits["airmass"].values,marker="+",c="b",lw=0)
# plt.plot(df_visits["exp_midpt"].values,df_visits["airmass"].values,marker="+",c="b",lw=0)
# Set x-axis to show dates
# plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d %H:%M:%S'))
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter("%Y-%m-%dT%H"))
plt.gca().xaxis.set_major_locator(mdates.AutoDateLocator())

# Rotate and format x-axis labels for readability
# ax.set_xticks(rotation=45, ha="right")
# ax.set_ylim(2,0.9)
ax.set_xlabel("Time")
ax.set_ylabel("dimm seeing")
ax.set_title(f"Dimm seeing vs Time")
ax.grid()

plt.show()

### Focus_z vs time

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12, 4), layout="constrained")

df_visits.plot(x="exp_midpt", y="focus_z", ax=ax, marker=".", lw=0, color="b")
# plt.plot(df_visits["exp_midpt"].values,df_visits["airmass"].values,marker="+",c="b",lw=0)
# plt.plot(df_visits["exp_midpt"].values,df_visits["airmass"].values,marker="+",c="b",lw=0)
# Set x-axis to show dates
# plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d %H:%M:%S'))
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter("%Y-%m-%dT%H"))
plt.gca().xaxis.set_major_locator(mdates.AutoDateLocator())

# Rotate and format x-axis labels for readability
# ax.set_xticks(rotation=45, ha="right")
# ax.set_ylim(2,0.9)
ax.set_xlabel("Time")
ax.set_ylabel("focus_z")
ax.set_title("Focus_z vs Time")
ax.grid()

plt.show()